In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.layers import ReLU, concatenate
path='/content/drive/MyDrive/assignment 1/annotations.csv'
relative_path='/content/drive/MyDrive/assignment 1/train/train/'
from tensorflow.keras.optimizers import SGD

In [2]:
def load_data(path,relative_path):
    df=pd.read_csv(path)
    data=[]
    for filename in df['fname']:
        address=relative_path+filename
        data.append(np.load(address))
    label=df['label']
    return data,label


In [3]:
data,label=load_data(path,relative_path)


In [4]:
mx_len=0
mn_len=100000
tot=0
for spectrograms in data:
    mx_len=max(mx_len,spectrograms.shape[2])
    mn_len=min(mn_len,spectrograms.shape[2])
    tot+=spectrograms.shape[2]
tot/1000,mn_len,mx_len

(593.504, 29, 2584)

In [5]:
def list_to_np(data,max_len,label):
    final_data=[]
    labels=[]
    for spectrogram,l in zip(data,label):
        x=spectrogram.flatten()
        k=len(x)/128
        if k>600:
          k=(int)(k/600)+1
          for i in range (k):
            if i!=k-1:
              final_data.append(x[i*600*128:(i+1)*600*128])
              labels.append(l)
            else:
              y=np.zeros((600*k*128-len(x)))
              y=np.append(x[(k-1)*600*128:],y)
              final_data.append(y)
              labels.append(l)
        else:
            y=np.zeros((600*128-len(x)))
            y=np.append(x,y)
            final_data.append(y)
            labels.append(l)
    return np.array(final_data),labels

In [6]:

data,label=list_to_np(data,mx_len,label)

In [7]:
def mfcc(data,mean=1):
  k=0
  if mean!=0:
    p=np.zeros((len(data),128))
  else:
    p=np.zeros((len(data),128,151))
  for i in range(len(data)):
    mfcc_feature=librosa.feature.mfcc(y=data[i],sr=44100,n_fft=2048,hop_length=512,n_mfcc=128,window='hann')  
    k=max(k,np.max(mfcc_feature))
    
    if mean!=0:
      mfcc_feature=np.mean(mfcc_feature,axis=1)    
      mfcc_feature=mfcc_feature.flatten()
      mfcc_feature=np.array(mfcc_feature)      
    p[i]=mfcc_feature
  p=p/k
  return p

In [8]:
label=pd.get_dummies(label)

In [9]:
mfcc_data=mfcc(data,0)

In [10]:


def d1(data):
  d=[]
  for i in range(len(data)):
    k=data[i].flatten()
    k=np.reshape(k,(128,151,1))
  
    d.append(k)
  d=np.array(d)
  return d
d=d1(mfcc_data)
train_x,test_x,train_y,test_y=train_test_split(d,label,test_size=0.2,random_state=711)
del d
del data

In [11]:
model = Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(128,151,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [12]:
opt = SGD(learning_rate=0.01, momentum=0.9)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
 history = model.fit(train_x, train_y, epochs=70,batch_size=32)

Epoch 1/70
40/40 [==============================] - 4s 25ms/step - loss: 2.2916 - accuracy: 0.1359
Epoch 2/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2875 - accuracy: 0.1383
Epoch 3/70
40/40 [==============================] - 1s 24ms/step - loss: 2.2858 - accuracy: 0.1328
Epoch 4/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2799 - accuracy: 0.1367
Epoch 5/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2756 - accuracy: 0.1359
Epoch 6/70
40/40 [==============================] - 1s 24ms/step - loss: 2.2670 - accuracy: 0.1703
Epoch 7/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2567 - accuracy: 0.1445
Epoch 8/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2494 - accuracy: 0.1703
Epoch 9/70
40/40 [==============================] - 1s 24ms/step - loss: 2.2239 - accuracy: 0.1891
Epoch 10/70
40/40 [==============================] - 1s 25ms/step - loss: 2.2068 - accuracy: 0.2070
Epoch 11/

In [14]:
model.evaluate(test_x,test_y,verbose=2)

11/11 - 0s - loss: 1.7529 - accuracy: 0.7040 - 302ms/epoch - 27ms/step


[1.752852439880371, 0.704049825668335]